In [1]:
import sys
import os
from copy import deepcopy

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping
import wandb

sys.path.append('..')

from src.configs import DataConfig, NetConfig, NetArchitecture, FilterConfig, DataType, CNNConfig, LC_SIZE
from src.utils import train, get_wandb_logger
from src.configs import PACKAGE_PATH
from src.data.data_processor import DataProcessor
from src.sweeps.sweep import DATA_CONFIG
from src.module.lightning_module import LCModule
from src.data.augmentations import compose, CyclicShift

import torch
import numpy as np
import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


In [6]:
data_cfg: DataConfig = deepcopy(DATA_CONFIG)
data_cfg.validation_path = f"{PACKAGE_PATH}/resources/SDLCD.csv"
data_cfg.data_types = [DataType.LC]
data_cfg.train_augmentations = compose([CyclicShift()])

net_cfg = NetConfig(
    name="CNN_sweep", input_size=LC_SIZE, 
    class_names=data_cfg.class_names,
    output_size=len(data_cfg.class_names),
    architecture=NetArchitecture.CNN,
    learning_rate=0.001,
    label_smoothing=0.,
    use_mixup=False,
    args=CNNConfig(input_size=LC_SIZE, output_size=5, in_channels=1, 
                conv_layers=[(16,7, 5),(32, 3, 1)], 
                classifier_layers=[])
)

In [7]:
dp = DataProcessor(data_cfg)

if os.path.exists(f'{dp.output_path}/{dp.hash}'):
    dp.load_data_from_file()
    dp.load_data_from_file(test=True)
else:
    dp.create_dataset_from_csv()
    dp.save_data(test=True)
    dp.save_data()


In [8]:
from src.utils import get_wandb_logger

# logger = get_wandb_logger("LC_TEST", "CNN3")
logger = None

In [9]:
net_cfg.input_size = dp.data_shape()
module = LCModule(net_cfg)

train(module, dp,
    num_epochs=3,
    batch_size=32,
    num_workers=4,
    callbacks=[EarlyStopping(monitor='val_loss', mode='min', patience=15)],
    sampler=False,
    max_num_samples=20_000,
    logger=logger)

CNN middle dim: 480


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | net       | CNN              | 4.2 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 548/548 [00:04<00:00, 134.96it/s, v_num=9, train_loss=0.648, train_acc=0.781, val_loss=1.100, val_acc=0.629]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 548/548 [00:04<00:00, 134.70it/s, v_num=9, train_loss=0.648, train_acc=0.781, val_loss=1.100, val_acc=0.629]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 62.73it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6145833134651184
        test_loss           1.1036823987960815
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
